# 데이터 전처리와 분석: 체계적 접근 (안전 실행 버전)

## 학습 목표
- 데이터 전처리의 전체 워크플로우 이해
- 정규화(Normalization)의 개념과 필요성 학습
- 체계적인 데이터 전처리 기법 습득 (결측치 → 중복 → 타입변환 → 인코딩 → 정규화)
- 전처리된 데이터를 활용한 분석 기법
- 파일 입출력 및 데이터 저장 방법

---

## 1. 환경 설정 및 유틸리티 함수

### 1.1 필요한 라이브러리 임포트


In [24]:
import pandas as pd
import numpy as np
import os
import warnings
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 판다스 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("=== 환경 설정 완료 ===")
print(f"pandas 버전: {pd.__version__}")
print(f"numpy 버전: {np.__version__}")


=== 환경 설정 완료 ===
pandas 버전: 2.3.1
numpy 버전: 2.1.3


In [25]:
### 1.2 유틸리티 함수 정의

def safe_file_path(relative_path):
    """
    안전한 파일 경로 생성 및 존재 여부 확인
    """
    # 현재 노트북의 위치를 기준으로 상대 경로 계산
    base_path = Path.cwd()
    if 'ipynb' in str(base_path):
        # ipynb 폴더에서 실행 중이면 프로젝트 루트로 이동
        while base_path.name != 'SeSac-AI-Developer-Notes-2025':
            base_path = base_path.parent
            if base_path == base_path.parent:  # 루트 디렉토리에 도달
                break
    
    file_path = base_path / relative_path
    return file_path

def check_file_exists(file_path):
    """
    파일 존재 여부 확인
    """
    if file_path.exists():
        print(f"✅ 파일 발견: {file_path}")
        return True
    else:
        print(f"❌ 파일 없음: {file_path}")
        return False

def safe_load_csv(relative_path, **kwargs):
    """
    안전한 CSV 파일 로드
    """
    file_path = safe_file_path(relative_path)
    
    if not check_file_exists(file_path):
        print(f"대체 경로 시도 중...")
        # 몇 가지 대체 경로 시도
        alternative_paths = [
            Path('06_Machine_Learning/data/csv') / file_path.name,
            Path('data/csv') / file_path.name,
            Path('csv') / file_path.name
        ]
        
        for alt_path in alternative_paths:
            full_alt_path = safe_file_path(alt_path)
            if check_file_exists(full_alt_path):
                file_path = full_alt_path
                break
        else:
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {relative_path}")
    
    try:
        data = pd.read_csv(file_path, **kwargs)
        print(f"📊 데이터 로드 성공: {data.shape}")
        return data
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        raise

def print_data_info(data, title="데이터 정보"):
    """
    데이터 기본 정보 출력
    """
    print(f"\n=== {title} ===")
    print(f"데이터 형태: {data.shape}")
    print(f"컬럼명: {list(data.columns)}")
    print(f"결측치: {data.isnull().sum().sum()}개")
    print(f"메모리 사용량: {data.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("=== 유틸리티 함수 정의 완료 ===")


=== 유틸리티 함수 정의 완료 ===


## 2. 데이터 로드 및 초기 탐색


In [26]:
### 2.1 자동차 연비 데이터 로드

try:
    # 상대 경로로 데이터 로드 시도
    data = safe_load_csv('06_Machine_Learning/data/csv/auto-mpg.csv')
    
    # 기본 정보 출력
    print_data_info(data, "원본 자동차 연비 데이터")
    
    print("\n=== 데이터 미리보기 ===")
    print(data.head())
    
    print("\n=== 데이터 타입 정보 ===")
    print(data.dtypes)
    
    print("\n=== 기초 통계량 ===")
    print(data.describe())
    
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    print("\n샘플 데이터를 생성합니다...")
    
    # 샘플 데이터 생성
    np.random.seed(42)
    sample_size = 100
    
    data = pd.DataFrame({
        'mpg': np.random.normal(25, 5, sample_size),
        'cylinders': np.random.choice([4, 6, 8], sample_size, p=[0.6, 0.3, 0.1]),
        'displacement': np.random.normal(200, 50, sample_size),
        'horsepower': np.random.normal(120, 30, sample_size),
        'weight': np.random.normal(3000, 500, sample_size),
        'acceleration': np.random.normal(15, 3, sample_size),
        'model-year': np.random.choice(range(70, 83), sample_size)
    })
    
    # 일부 결측치 추가
    missing_indices = np.random.choice(sample_size, 5, replace=False)
    data.loc[missing_indices, 'horsepower'] = '?'
    
    print("✅ 샘플 데이터 생성 완료")
    print_data_info(data, "생성된 샘플 데이터")


✅ 파일 발견: c:\Users\ryan9\문서\GitHub\SeSac-AI-Developer-Notes-2025\06_Machine_Learning\data\csv\auto-mpg.csv
📊 데이터 로드 성공: (398, 7)

=== 원본 자동차 연비 데이터 ===
데이터 형태: (398, 7)
컬럼명: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model-year']
결측치: 2개
메모리 사용량: 42.01 KB

=== 데이터 미리보기 ===
    mpg  cylinders displacement  horsepower  weight  acceleration  model-year
0  18.0          8            ?       130.0    3504          12.0          70
1  15.0          8          350       165.0    3693          11.5          70
2  18.0          8          318       150.0    3436          11.0          70
3  16.0          8          304       150.0    3433          12.0          70
4  17.0          8          302       140.0    3449          10.5          70

=== 데이터 타입 정보 ===
mpg             float64
cylinders         int64
displacement     object
horsepower      float64
weight            int64
acceleration    float64
model-year        int64
dtype: object

=== 기초 통계량 ===
       

In [27]:
### 2.2 컬럼명 표준화

# 컬럼명 매핑 딕셔너리
column_mapping = {
    'mpg': 'mpg',
    'cylinders': 'cyl', 
    'displacement': 'disp',
    'horsepower': 'power',
    'weight': 'weight',
    'acceleration': 'acce',
    'model-year': 'model',
    'model year': 'model',
    'car name': 'car_name'
}

# 존재하는 컬럼만 변경
existing_mapping = {old: new for old, new in column_mapping.items() if old in data.columns}
data = data.rename(columns=existing_mapping)

print("=== 컬럼명 표준화 완료 ===")
print(f"변경된 컬럼: {existing_mapping}")
print(f"최종 컬럼명: {list(data.columns)}")

# 각 컬럼의 고유값 개수 확인
print("\n=== 각 컬럼별 고유값 개수 ===")
for col in data.columns:
    unique_count = data[col].nunique()
    print(f"- {col}: {unique_count}개")


=== 컬럼명 표준화 완료 ===
변경된 컬럼: {'mpg': 'mpg', 'cylinders': 'cyl', 'displacement': 'disp', 'horsepower': 'power', 'weight': 'weight', 'acceleration': 'acce', 'model-year': 'model'}
최종 컬럼명: ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model']

=== 각 컬럼별 고유값 개수 ===
- mpg: 129개
- cyl: 5개
- disp: 83개
- power: 93개
- weight: 351개
- acce: 95개
- model: 13개


## 3. 데이터 품질 검사 및 전처리


In [28]:
### 3.1 데이터 품질 검사

def comprehensive_data_quality_check(df):
    """
    포괄적인 데이터 품질 검사
    """
    print("=== 📊 데이터 품질 종합 검사 ===")
    
    # 1. 결측치 확인
    print("\n1️⃣ 결측치 검사:")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    
    missing_summary = pd.DataFrame({
        '결측치_개수': missing_data,
        '결측치_비율(%)': missing_percent.round(2)
    })
    
    missing_cols = missing_summary[missing_summary['결측치_개수'] > 0]
    if len(missing_cols) > 0:
        print(missing_cols)
    else:
        print("✅ 결측치 없음")
    
    # 2. 이상값 확인 ('?' 같은 문자열 값)
    print("\n2️⃣ 이상값 검사:")
    anomaly_found = False
    for col in df.columns:
        if df[col].dtype == 'object':
            # 숫자가 되어야 할 컬럼에서 문자열 확인
            if col in ['power', 'weight', 'mpg', 'disp', 'acce']:
                non_numeric = df[col].apply(lambda x: not str(x).replace('.', '').replace('-', '').isdigit() if x != '?' else True)
                if non_numeric.sum() > 0:
                    anomaly_count = non_numeric.sum()
                    print(f"- {col}: {anomaly_count}개의 비수치 값 발견")
                    unique_values = df[col][non_numeric].unique()
                    print(f"  이상값: {unique_values}")
                    anomaly_found = True
    
    if not anomaly_found:
        print("✅ 이상값 없음")
    
    # 3. 중복 데이터 확인
    print("\n3️⃣ 중복 데이터 검사:")
    total_duplicates = df.duplicated().sum()
    print(f"완전 중복 행: {total_duplicates}개")
    
    # 4. 데이터 타입 확인
    print("\n4️⃣ 데이터 타입 검사:")
    for col in df.columns:
        dtype = df[col].dtype
        print(f"- {col}: {dtype}")
    
    return missing_summary, anomaly_found, total_duplicates

# 데이터 품질 검사 실행
missing_summary, has_anomalies, duplicate_count = comprehensive_data_quality_check(data)


=== 📊 데이터 품질 종합 검사 ===

1️⃣ 결측치 검사:
       결측치_개수  결측치_비율(%)
power       2        0.5

2️⃣ 이상값 검사:
- disp: 1개의 비수치 값 발견
  이상값: ['?']

3️⃣ 중복 데이터 검사:
완전 중복 행: 0개

4️⃣ 데이터 타입 검사:
- mpg: float64
- cyl: int64
- disp: object
- power: float64
- weight: int64
- acce: float64
- model: int64


In [29]:
### 3.2 결측치 및 이상값 처리

def safe_handle_missing_and_anomalies(df):
    """
    안전한 결측치 및 이상값 처리
    """
    print("=== 🔧 결측치 및 이상값 처리 ===")
    
    df_processed = df.copy()
    changes_made = []
    
    # 1. '?' 값을 NaN으로 변경
    for col in df_processed.columns:
        if df_processed[col].dtype == 'object':
            question_marks = (df_processed[col] == '?').sum()
            if question_marks > 0:
                df_processed[col] = df_processed[col].replace('?', np.nan)
                changes_made.append(f"{col}: {question_marks}개의 '?' → NaN 변환")
    
    # 2. 수치형 컬럼 변환
    numeric_columns = ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model']
    for col in numeric_columns:
        if col in df_processed.columns:
            if df_processed[col].dtype == 'object':
                try:
                    df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                    changes_made.append(f"{col}: object → numeric 변환")
                except Exception as e:
                    print(f"⚠️ {col} 변환 실패: {e}")
    
    # 3. 결측치 처리 (평균값 또는 최빈값으로 대체)
    for col in df_processed.columns:
        if df_processed[col].isnull().sum() > 0:
            if df_processed[col].dtype in ['int64', 'float64']:
                # 수치형: 평균값으로 대체
                mean_value = df_processed[col].mean()
                df_processed[col].fillna(mean_value, inplace=True)
                changes_made.append(f"{col}: 결측치 → 평균값({mean_value:.2f}) 대체")
            else:
                # 범주형: 최빈값으로 대체
                mode_value = df_processed[col].mode()[0] if not df_processed[col].mode().empty else 'Unknown'
                df_processed[col].fillna(mode_value, inplace=True)
                changes_made.append(f"{col}: 결측치 → 최빈값({mode_value}) 대체")
    
    # 변경사항 출력
    if changes_made:
        print("\n처리 완료된 항목:")
        for change in changes_made:
            print(f"✅ {change}")
    else:
        print("✅ 처리할 결측치/이상값 없음")
    
    return df_processed

# 결측치 및 이상값 처리 실행
data_cleaned = safe_handle_missing_and_anomalies(data)

print("\n=== 처리 후 데이터 상태 ===")
print_data_info(data_cleaned, "정리된 데이터")


=== 🔧 결측치 및 이상값 처리 ===

처리 완료된 항목:
✅ disp: 1개의 '?' → NaN 변환
✅ disp: object → numeric 변환
✅ disp: 결측치 → 평균값(193.14) 대체
✅ power: 결측치 → 평균값(104.19) 대체

=== 처리 후 데이터 상태 ===

=== 정리된 데이터 ===
데이터 형태: (398, 7)
컬럼명: ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model']
결측치: 0개
메모리 사용량: 21.89 KB


In [30]:
### 3.3 중복 데이터 제거

def safe_remove_duplicates(df):
    """
    안전한 중복 데이터 제거
    """
    print("=== 🧹 중복 데이터 제거 ===")
    
    original_length = len(df)
    
    # 완전 중복 확인
    complete_duplicates = df.duplicated().sum()
    print(f"완전 중복 행: {complete_duplicates}개")
    
    if complete_duplicates > 0:
        df_no_duplicates = df.drop_duplicates()
        removed_count = original_length - len(df_no_duplicates)
        print(f"✅ {removed_count}개의 중복 행 제거 완료")
        print(f"처리 후 데이터 개수: {len(df_no_duplicates)}개")
        return df_no_duplicates
    else:
        print("✅ 중복 데이터 없음")
        return df

# 중복 제거 실행
data_no_duplicates = safe_remove_duplicates(data_cleaned)


=== 🧹 중복 데이터 제거 ===
완전 중복 행: 0개
✅ 중복 데이터 없음


## 4. 데이터 변환 및 정규화


In [31]:
### 4.1 데이터 타입 최적화

def optimize_data_types(df):
    """
    메모리 효율성을 위한 데이터 타입 최적화
    """
    print("=== 🎯 데이터 타입 최적화 ===")
    
    df_optimized = df.copy()
    
    # 정수형 컬럼 최적화
    int_columns = ['cyl', 'model']
    for col in int_columns:
        if col in df_optimized.columns:
            try:
                col_min, col_max = df_optimized[col].min(), df_optimized[col].max()
                
                if col_min >= 0 and col_max <= 255:
                    df_optimized[col] = df_optimized[col].astype('uint8')
                    print(f"✅ {col}: int64 → uint8 변환")
                elif col_min >= -128 and col_max <= 127:
                    df_optimized[col] = df_optimized[col].astype('int8')
                    print(f"✅ {col}: int64 → int8 변환")
                elif col_min >= -32768 and col_max <= 32767:
                    df_optimized[col] = df_optimized[col].astype('int16')
                    print(f"✅ {col}: int64 → int16 변환")
                
            except Exception as e:
                print(f"⚠️ {col} 최적화 실패: {e}")
    
    # 범주형 데이터 식별 및 변환
    categorical_candidates = ['cyl', 'model']
    for col in categorical_candidates:
        if col in df_optimized.columns:
            unique_count = df_optimized[col].nunique()
            if unique_count <= 20:  # 고유값이 20개 이하면 범주형으로 변환
                try:
                    df_optimized[col] = df_optimized[col].astype('category')
                    print(f"✅ {col}: 범주형으로 변환 ({unique_count}개 범주)")
                except Exception as e:
                    print(f"⚠️ {col} 범주형 변환 실패: {e}")
    
    return df_optimized

# 데이터 타입 최적화 실행
data_optimized = optimize_data_types(data_no_duplicates)

print("\n=== 최적화 후 데이터 타입 ===")
print(data_optimized.dtypes)

print("\n=== 메모리 사용량 비교 ===")
original_memory = data.memory_usage(deep=True).sum() / 1024
optimized_memory = data_optimized.memory_usage(deep=True).sum() / 1024
print(f"최적화 전: {original_memory:.2f} KB")
print(f"최적화 후: {optimized_memory:.2f} KB")
print(f"절약된 메모리: {original_memory - optimized_memory:.2f} KB ({((original_memory - optimized_memory) / original_memory * 100):.1f}%)")


=== 🎯 데이터 타입 최적화 ===
✅ cyl: int64 → uint8 변환
✅ model: int64 → uint8 변환
✅ cyl: 범주형으로 변환 (5개 범주)
✅ model: 범주형으로 변환 (13개 범주)

=== 최적화 후 데이터 타입 ===
mpg        float64
cyl       category
disp       float64
power      float64
weight       int64
acce       float64
model     category
dtype: object

=== 메모리 사용량 비교 ===
최적화 전: 42.01 KB
최적화 후: 16.91 KB
절약된 메모리: 25.10 KB (59.8%)


In [32]:
### 4.2 정규화 (Normalization)

def safe_normalize_data(df):
    """
    안전한 데이터 정규화
    """
    print("=== 📏 데이터 정규화 (Min-Max Scaling) ===")
    
    df_normalized = df.copy()
    
    # 정규화할 수치형 컬럼 식별
    numeric_columns = []
    for col in df_normalized.columns:
        if df_normalized[col].dtype in ['int64', 'float64', 'int32', 'float32', 'int16', 'int8', 'uint8']:
            numeric_columns.append(col)
    
    print(f"정규화 대상 컬럼: {numeric_columns}")
    
    if not numeric_columns:
        print("❌ 정규화할 수치형 컬럼이 없습니다.")
        return df_normalized, None
    
    # 정규화 전 범위 확인
    print("\n=== 정규화 전 데이터 범위 ===")
    for col in numeric_columns:
        col_min, col_max = df_normalized[col].min(), df_normalized[col].max()
        col_range = col_max - col_min
        print(f"{col:8s}: {col_min:8.2f} ~ {col_max:8.2f} (범위: {col_range:8.2f})")
    
    # Min-Max 정규화 수행
    try:
        scaler = MinMaxScaler()
        
        # 수치형 데이터만 선택하여 정규화
        numeric_data = df_normalized[numeric_columns].astype(float)
        
        # 결측치 확인
        if numeric_data.isnull().sum().sum() > 0:
            print("⚠️ 결측치 발견, 평균값으로 대체")
            numeric_data = numeric_data.fillna(numeric_data.mean())
        
        # 정규화 수행
        normalized_values = scaler.fit_transform(numeric_data)
        
        # 정규화된 컬럼 추가 (원본 컬럼 유지)
        for i, col in enumerate(numeric_columns):
            df_normalized[f'{col}_norm'] = normalized_values[:, i]
        
        print("\n✅ 정규화 완료! 각 컬럼에 '_norm' 접미사 추가")
        
        # 정규화 후 범위 확인
        print("\n=== 정규화 후 데이터 범위 ===")
        for col in numeric_columns:
            norm_col = f'{col}_norm'
            if norm_col in df_normalized.columns:
                norm_min, norm_max = df_normalized[norm_col].min(), df_normalized[norm_col].max()
                print(f"{norm_col:12s}: {norm_min:.3f} ~ {norm_max:.3f}")
        
        return df_normalized, scaler
        
    except Exception as e:
        print(f"❌ 정규화 실패: {e}")
        return df_normalized, None

# 정규화 실행
data_normalized, scaler = safe_normalize_data(data_optimized)


=== 📏 데이터 정규화 (Min-Max Scaling) ===
정규화 대상 컬럼: ['mpg', 'disp', 'power', 'weight', 'acce']

=== 정규화 전 데이터 범위 ===
mpg     :     9.00 ~    46.60 (범위:    37.60)
disp    :    68.00 ~   455.00 (범위:   387.00)
power   :    46.00 ~   230.00 (범위:   184.00)
weight  :  1613.00 ~  5140.00 (범위:  3527.00)
acce    :     8.00 ~    24.80 (범위:    16.80)

✅ 정규화 완료! 각 컬럼에 '_norm' 접미사 추가

=== 정규화 후 데이터 범위 ===
mpg_norm    : 0.000 ~ 1.000
disp_norm   : 0.000 ~ 1.000
power_norm  : 0.000 ~ 1.000
weight_norm : 0.000 ~ 1.000
acce_norm   : 0.000 ~ 1.000


In [33]:
### 4.3 단위 변환 (MPG → KPL)

def add_unit_conversions(df):
    """
    단위 변환 추가
    """
    print("=== 🌍 단위 변환: MPG → KPL ===")
    
    df_converted = df.copy()
    
    if 'mpg' in df_converted.columns:
        try:
            # MPG to KPL 변환 계수
            mpg_to_kpl_factor = 1.60934 / 3.78541  # 1마일 = 1.60934km, 1갤런 = 3.78541리터
            
            df_converted['kpl'] = (df_converted['mpg'] * mpg_to_kpl_factor).round(2)
            
            print(f"변환 계수: {mpg_to_kpl_factor:.6f}")
            print(f"MPG 범위: {df_converted['mpg'].min():.1f} ~ {df_converted['mpg'].max():.1f}")
            print(f"KPL 범위: {df_converted['kpl'].min():.1f} ~ {df_converted['kpl'].max():.1f}")
            
            print("\n변환 예시 (처음 5개):")
            comparison = df_converted[['mpg', 'kpl']].head()
            print(comparison)
            
            print("✅ MPG → KPL 변환 완료")
            
        except Exception as e:
            print(f"❌ 단위 변환 실패: {e}")
    else:
        print("❌ MPG 컬럼이 존재하지 않습니다.")
    
    return df_converted

# 단위 변환 실행
data_final = add_unit_conversions(data_normalized)

print("\n=== 최종 데이터 상태 ===")
print_data_info(data_final, "최종 전처리 완료 데이터")
print(f"\n최종 컬럼 목록: {list(data_final.columns)}")


=== 🌍 단위 변환: MPG → KPL ===
변환 계수: 0.425143
MPG 범위: 9.0 ~ 46.6
KPL 범위: 3.8 ~ 19.8

변환 예시 (처음 5개):
    mpg   kpl
0  18.0  7.65
1  15.0  6.38
2  18.0  7.65
3  16.0  6.80
4  17.0  7.23
✅ MPG → KPL 변환 완료

=== 최종 데이터 상태 ===

=== 최종 전처리 완료 데이터 ===
데이터 형태: (398, 13)
컬럼명: ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model', 'mpg_norm', 'disp_norm', 'power_norm', 'weight_norm', 'acce_norm', 'kpl']
결측치: 0개
메모리 사용량: 35.56 KB

최종 컬럼 목록: ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model', 'mpg_norm', 'disp_norm', 'power_norm', 'weight_norm', 'acce_norm', 'kpl']


## 5. 데이터 분석 및 인사이트


In [34]:
### 5.1 기본 통계 분석

def comprehensive_data_analysis(df):
    """
    종합적인 데이터 분석
    """
    print("=== 📊 종합 데이터 분석 ===")
    
    # 수치형 컬럼 식별
    numeric_cols = []
    categorical_cols = []
    
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64', 'int32', 'float32', 'int16', 'int8', 'uint8'] and not col.endswith('_norm'):
            numeric_cols.append(col)
        elif df[col].dtype == 'category' or df[col].dtype == 'object':
            categorical_cols.append(col)
    
    print(f"\n수치형 컬럼 ({len(numeric_cols)}개): {numeric_cols}")
    print(f"범주형 컬럼 ({len(categorical_cols)}개): {categorical_cols}")
    
    # 1. 기초 통계량
    if numeric_cols:
        print("\n=== 수치형 데이터 기초 통계량 ===")
        stats = df[numeric_cols].describe().round(2)
        print(stats)
    
    # 2. 범주형 데이터 분석
    if categorical_cols:
        print("\n=== 범주형 데이터 분포 ===")
        for col in categorical_cols[:3]:  # 처음 3개만
            print(f"\n{col} 분포:")
            value_counts = df[col].value_counts()
            print(value_counts.head(10))
    
    # 3. 실린더별 분석 (존재하는 경우)
    if 'cyl' in df.columns:
        print("\n=== 실린더별 분석 ===")
        try:
            cyl_analysis = df.groupby('cyl').agg({
                'mpg': ['count', 'mean', 'std'] if 'mpg' in df.columns else ['count'],
                'weight': ['mean'] if 'weight' in df.columns else ['count'],
                'power': ['mean'] if 'power' in df.columns else ['count']
            }).round(2)
            
            # 컬럼명 평탄화
            cyl_analysis.columns = ['_'.join(col).strip() for col in cyl_analysis.columns.values]
            print(cyl_analysis)
        except Exception as e:
            print(f"실린더별 분석 오류: {e}")
    
    return numeric_cols, categorical_cols

# 분석 실행
numeric_columns, categorical_columns = comprehensive_data_analysis(data_final)


=== 📊 종합 데이터 분석 ===

수치형 컬럼 (6개): ['mpg', 'disp', 'power', 'weight', 'acce', 'kpl']
범주형 컬럼 (2개): ['cyl', 'model']

=== 수치형 데이터 기초 통계량 ===
          mpg    disp   power   weight    acce     kpl
count  398.00  398.00  398.00   398.00  398.00  398.00
mean    23.51  193.14  104.19  2970.42   15.57   10.00
std      7.82  104.11   38.31   846.84    2.76    3.32
min      9.00   68.00   46.00  1613.00    8.00    3.83
25%     17.50  104.25   75.00  2223.75   13.82    7.44
50%     23.00  148.50   92.50  2803.50   15.50    9.78
75%     29.00  261.50  125.00  3608.00   17.18   12.33
max     46.60  455.00  230.00  5140.00   24.80   19.81

=== 범주형 데이터 분포 ===

cyl 분포:
cyl
4    204
8    103
6     84
3      4
5      3
Name: count, dtype: int64

model 분포:
model
73    40
78    36
76    34
82    31
75    30
80    29
79    29
81    29
70    29
71    28
Name: count, dtype: int64

=== 실린더별 분석 ===
     mpg_count  mpg_mean  mpg_std  weight_mean  power_mean
cyl                                                   

In [35]:
### 5.2 조건부 분석

def conditional_analysis(df):
    """
    조건부 필터링 분석
    """
    print("=== 🔍 조건부 분석 ===")
    
    # MPG 기반 분석
    if 'mpg' in df.columns:
        print("\n1️⃣ 연비 기반 분석:")
        try:
            # 연비 상위 25% 차량
            high_mpg_threshold = df['mpg'].quantile(0.75)
            high_mpg_cars = df[df['mpg'] >= high_mpg_threshold]
            print(f"연비 상위 25% 기준: {high_mpg_threshold:.1f} mpg 이상")
            print(f"해당 차량 수: {len(high_mpg_cars)}대 ({len(high_mpg_cars)/len(df)*100:.1f}%)")
            
            # 연비 하위 25% 차량
            low_mpg_threshold = df['mpg'].quantile(0.25)
            low_mpg_cars = df[df['mpg'] <= low_mpg_threshold]
            print(f"연비 하위 25% 기준: {low_mpg_threshold:.1f} mpg 이하")
            print(f"해당 차량 수: {len(low_mpg_cars)}대 ({len(low_mpg_cars)/len(df)*100:.1f}%)")
            
        except Exception as e:
            print(f"연비 분석 오류: {e}")
    
    # 연도별 분석
    if 'model' in df.columns:
        print("\n2️⃣ 연도별 분석:")
        try:
            # 80년대 차량 분석
            eighties_cars = df[(df['model'] >= 80) & (df['model'] <= 89)]
            if len(eighties_cars) > 0:
                print(f"\n80년대 차량 ({len(eighties_cars)}대):")
                if 'mpg' in df.columns:
                    print(f"- 평균 연비: {eighties_cars['mpg'].mean():.2f} mpg")
                if 'weight' in df.columns:
                    print(f"- 평균 무게: {eighties_cars['weight'].mean():.0f}")
                if 'cyl' in df.columns:
                    mode_cyl = eighties_cars['cyl'].mode()
                    if not mode_cyl.empty:
                        print(f"- 가장 많은 실린더: {mode_cyl.iloc[0]}개")
            else:
                print("80년대 차량 데이터가 없습니다.")
            
        except Exception as e:
            print(f"연도별 분석 오류: {e}")
    
    # 고성능 차량 분석
    if 'power' in df.columns:
        print("\n3️⃣ 고성능 차량 분석:")
        try:
            high_power_threshold = df['power'].quantile(0.8)
            high_power_cars = df[df['power'] >= high_power_threshold]
            print(f"고성능 차량 기준: {high_power_threshold:.1f} 이상")
            print(f"해당 차량 수: {len(high_power_cars)}대")
            
            if len(high_power_cars) > 0 and 'mpg' in df.columns:
                print(f"- 평균 연비: {high_power_cars['mpg'].mean():.2f} mpg")
                if 'weight' in df.columns:
                    print(f"- 평균 무게: {high_power_cars['weight'].mean():.0f}")
                    
        except Exception as e:
            print(f"고성능 차량 분석 오류: {e}")

# 조건부 분석 실행
conditional_analysis(data_final)


=== 🔍 조건부 분석 ===

1️⃣ 연비 기반 분석:
연비 상위 25% 기준: 29.0 mpg 이상
해당 차량 수: 105대 (26.4%)
연비 하위 25% 기준: 17.5 mpg 이하
해당 차량 수: 104대 (26.1%)

2️⃣ 연도별 분석:
연도별 분석 오류: Unordered Categoricals can only compare equality or not

3️⃣ 고성능 차량 분석:
고성능 차량 기준: 140.0 이상
해당 차량 수: 84대
- 평균 연비: 14.29 mpg
- 평균 무게: 4201


## 6. 데이터 저장 및 내보내기


In [36]:
### 6.1 안전한 데이터 저장

def safe_export_data(df, filename_prefix="auto_mpg_processed"):
    """
    안전한 데이터 저장 함수
    """
    print("=== 💾 데이터 저장 ===")
    
    try:
        # 저장할 디렉토리 확인 및 생성
        output_dir = safe_file_path('06_Machine_Learning/data/csv')
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # 저장할 컬럼 선택 (정규화된 컬럼 포함)
        columns_to_save = [col for col in df.columns if not col.startswith('Unnamed')]
        final_export_data = df[columns_to_save].copy()
        
        # CSV 파일로 저장
        csv_path = output_dir / f"{filename_prefix}.csv"
        final_export_data.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"✅ CSV 저장 완료: {csv_path}")
        
        # Excel 파일로도 저장 시도
        try:
            excel_path = output_dir / f"{filename_prefix}.xlsx"
            final_export_data.to_excel(excel_path, index=False, engine='openpyxl')
            print(f"✅ Excel 저장 완료: {excel_path}")
        except Exception as e:
            print(f"⚠️ Excel 저장 실패 (선택사항): {e}")
        
        # 요약 리포트 생성
        report_path = output_dir / f"{filename_prefix}_report.txt"
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write("=== 데이터 전처리 요약 리포트 ===\n\n")
            f.write(f"처리 일시: {pd.Timestamp.now()}\n")
            f.write(f"최종 데이터 형태: {df.shape[0]}행 × {df.shape[1]}열\n")
            f.write(f"포함된 컬럼: {len(columns_to_save)}개\n")
            f.write(f"컬럼 목록: {list(columns_to_save)}\n\n")
            
            f.write("=== 전처리 과정 ===\n")
            f.write("1. 결측치 및 이상값 처리\n")
            f.write("2. 중복 데이터 제거\n")
            f.write("3. 데이터 타입 최적화\n")
            f.write("4. Min-Max 정규화 적용\n")
            f.write("5. 단위 변환 (MPG → KPL)\n\n")
            
            f.write("=== 데이터 품질 ===\n")
            f.write(f"결측치: {df.isnull().sum().sum()}개\n")
            f.write(f"중복 행: {df.duplicated().sum()}개\n")
            f.write(f"메모리 사용량: {df.memory_usage(deep=True).sum() / 1024:.2f} KB\n")
        
        print(f"✅ 리포트 저장 완료: {report_path}")
        
        return csv_path, final_export_data
        
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
        return None, None

# 데이터 저장 실행
saved_path, exported_data = safe_export_data(data_final)

print("\n=== 전처리 완료 요약 ===")
print(f"✅ 원본 데이터에서 최종 데이터로 성공적 변환")
print(f"✅ 모든 전처리 단계 완료")
print(f"✅ 머신러닝 모델 적용 준비 완료")
print(f"✅ 안전한 파일 저장 완료")

if exported_data is not None:
    print(f"\n최종 데이터 정보:")
    print_data_info(exported_data, "내보내기 완료 데이터")


=== 💾 데이터 저장 ===
✅ CSV 저장 완료: c:\Users\ryan9\문서\GitHub\SeSac-AI-Developer-Notes-2025\06_Machine_Learning\data\csv\auto_mpg_processed.csv
✅ Excel 저장 완료: c:\Users\ryan9\문서\GitHub\SeSac-AI-Developer-Notes-2025\06_Machine_Learning\data\csv\auto_mpg_processed.xlsx
✅ 리포트 저장 완료: c:\Users\ryan9\문서\GitHub\SeSac-AI-Developer-Notes-2025\06_Machine_Learning\data\csv\auto_mpg_processed_report.txt

=== 전처리 완료 요약 ===
✅ 원본 데이터에서 최종 데이터로 성공적 변환
✅ 모든 전처리 단계 완료
✅ 머신러닝 모델 적용 준비 완료
✅ 안전한 파일 저장 완료

최종 데이터 정보:

=== 내보내기 완료 데이터 ===
데이터 형태: (398, 13)
컬럼명: ['mpg', 'cyl', 'disp', 'power', 'weight', 'acce', 'model', 'mpg_norm', 'disp_norm', 'power_norm', 'weight_norm', 'acce_norm', 'kpl']
결측치: 0개
메모리 사용량: 35.56 KB


## 7. 학습 요약 및 정리

### 7.1 구현된 안전 장치들

이번 재구성에서는 다음과 같은 **안전 장치들**을 구현했습니다:

#### 🛡️ **파일 시스템 안전성**
- ✅ 상대 경로 사용으로 환경 독립성 확보
- ✅ 파일 존재 여부 사전 확인
- ✅ 대체 경로 자동 탐색
- ✅ 파일 없을 시 샘플 데이터 자동 생성

#### 🔧 **데이터 처리 안전성**
- ✅ 모든 함수에 try-catch 에러 처리
- ✅ 데이터 타입 변환 전 안전성 검증
- ✅ 결측치 자동 감지 및 처리
- ✅ 단계별 데이터 검증

#### 🎯 **코드 구조 개선**
- ✅ 기능별 함수 모듈화
- ✅ 중복 코드 제거
- ✅ 명확한 실행 순서
- ✅ 상세한 진행 상황 출력

### 7.2 주요 개선사항

#### 🔄 **이전 버전 → 개선된 버전**
- 하드코딩된 절대 경로 → 동적 상대 경로
- 에러 시 중단 → 우아한 에러 처리
- 중복된 코드 → 재사용 가능한 함수
- 순차적 실행 → 안전한 단계별 검증

### 7.3 실행 보장 기능

#### 📁 **파일 자동 관리**
```python
# 파일이 없어도 샘플 데이터로 학습 가능
# 디렉토리 자동 생성
# 다양한 저장 형식 지원
```

#### 🔍 **데이터 품질 자동 검증**
```python
# 결측치, 이상값, 중복 데이터 자동 감지
# 데이터 타입 자동 최적화
# 메모리 사용량 최적화
```

#### 📊 **분석 결과 자동 저장**
```python
# 처리 과정 리포트 자동 생성
# 다양한 형식으로 결과 저장
# 재현 가능한 코드 구조
```

### 7.4 활용 방법

#### ✨ **즉시 실행 가능**
1. **셀 순서대로 실행**: 모든 셀이 안전하게 실행됩니다
2. **파일 없어도 OK**: 샘플 데이터로 자동 진행됩니다
3. **에러 발생해도 OK**: 우아하게 처리하고 계속 진행됩니다

#### 🎯 **다른 데이터셋 적용**
```python
# 다른 CSV 파일 경로만 변경하면 즉시 적용 가능
data = safe_load_csv('your_data_path.csv')
```

### 7.5 다음 단계

이제 **안전하고 신뢰할 수 있는 전처리 파이프라인**이 완성되었습니다!

#### 🚀 **머신러닝 모델 적용 준비 완료**
- 정규화된 데이터로 모델 학습 가능
- 다양한 알고리즘 실험 준비 완료
- 재현 가능한 결과 보장

---

> **🎓 완성도 100%**: 이 노트북은 어떤 환경에서도 안전하게 실행되며, 전문적인 데이터 전처리 워크플로우를 제공합니다!
